In [ ]:
import whisperx
import json
import spacy
import tensorflow as tf
import torch
import os


gpu = tf.config.list_physical_devices('GPU')

DEVICE = "cuda" if len(gpu) >= 1 else "cpu"

#pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
dossier = "whisperX"
ad = "ad"
cn = "cn"

torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

silences = ['.', '..', 'long silence'] # 2-4 sec / 4-6 sec / more than 6

DEVICE

'cuda'

In [ ]:
model = whisperx.load_model("medium.en", DEVICE)
#whisperx_result = model.transcribe("adrso025.wav")
#whisperx_result['text']

In [ ]:
def save_json(result, dossier, name):
    with open(dossier + "/"+ name, "w") as f:
        json.dump(result, f)
    return True

def white_spaces_insert(list_dict):
    text_with_whitespaces = [] 
    for i in range(0, len(list_dict) - 1):
        text_with_whitespaces.append(list_dict[i])
        whitespace_lenght = list_dict[i+1]['start'] - list_dict[i]['end'] 
        if (whitespace_lenght >= 2) and (whitespace_lenght < 4) :
            text_with_whitespaces.append({'text': silences[0]})
        if (whitespace_lenght >= 4) and (whitespace_lenght < 6) :
            text_with_whitespaces.append({'text': silences[1]})
        if (whitespace_lenght >= 6):
            text_with_whitespaces.append({'text': silences[2]})
    return text_with_whitespaces

def do_transcriptions_in_folder(folder_path, dossier, label):
    #create folders
    u_dossier = dossier + "/" + label
    try:
        os.makedirs(u_dossier)
    except FileExistsError:
        pass
    
    white_space_folder = dossier + "/" + "white_spaces_" + label
    try:
        os.makedirs(white_space_folder)
    except FileExistsError:
        pass
    
    #list of audios name
    files = [file for file in os.listdir(folder_path) if (os.path.isfile(os.path.join(folder_path, file) and file.endswith(".wav")))]
    
    for audio in files:
        #do transcription
        name_audio = folder_path + '/' + audio
        whisperx_result = model.transcribe(name_audio)
        # load alignment model and metadata
        model_a, metadata = whisperx.load_align_model(language_code=whisperx_result["language"], device=DEVICE)
        # align whisper output
        result_aligned = whisperx.align(whisperx_result["segments"], model_a, metadata, name_audio, DEVICE)
        #save in json
        res = {}
        res["text"] = concatenate_text(result_aligned["word_segments"])
        res["label"] = 1 if label == "ad" else 0
        save_json(res, u_dossier, audio.replace(".wav",".json"))
        
        #with_white_spaces
        dict_with_white_spaces = {}
        white_space = white_spaces_insert(result_aligned["word_segments"])
        #insert key "text" (all text)
        all_text = concatenate_text(white_space)
        dict_with_white_spaces["text"] = all_text
        #insert key "label" = 0 if cn 1 if ad
        dict_with_white_spaces["label"] = 1 if label == "ad" else 0
        save_json(dict_with_white_spaces, white_space_folder, audio.replace(".wav",".json"))
     
    return True

def concatenate_text(dictionnaries):
    text = ""
    if len(dictionnaries) >= 1:
        for i, dictionnary in enumerate(dictionnaries[:-1]):        
            text += dictionnary["text"] + " "
        #last word (presumed '.')
        text += dictionnaries[-1]["text"]
        return text
    else :
        return text
    

In [ ]:
do_transcriptions_in_folder("ad", dossier, "ad")
do_transcriptions_in_folder("cn", dossier, "cn")



In [ ]:
do_transcriptions_in_folder("cn_segmented", "whisperX_seg", "cn")
do_transcriptions_in_folder("ad_segmented", "whisperX_seg", "ad")

[00:00.703 --> 00:05.460]  Okay, I see a mom doing the dishes and the water's overflowing from the sink and
[00:05.560 --> 00:14.140]  she seems to be completely oblivious of it. The children in the picture, who I
[00:14.561 --> 00:20.540]  presume are hers, are climbing, one of the boys climbing on a, or standing on a
[00:21.443 --> 00:26.400]  stool that's about to fall over, putting his hand in the cookie jar on the top
[00:26.601 --> 00:30.000]  shelf and his sister or cousin or whoever.
[00:00.000 --> 00:02.878]  whoever the girl is, is reaching up for a cookie.
[00:04.580 --> 00:07.495]  The window is open, there's a pathway
[00:08.460 --> 00:12.256]  with garden on either side, it looks like.
[00:13.221 --> 00:15.317]  There's something else on the sink,
[00:15.881 --> 00:18.376]  I mean on the countertop, dishes.
[00:20.921 --> 00:23.835]  And that's, the mother has an apron on.
[00:25.021 --> 00:29.139]  She is dressed in the Mrs. Beaver style
[00:29.320 --> 00:29.629]  with t

[00:00.040 --> 00:05.458]  Hmm, a lot of fall and safety problems.
[00:06.060 --> 00:08.659]  So this boy is getting cookies from a cookie jar,
[00:09.081 --> 00:11.294]  and the stool is tipping.
[00:12.280 --> 00:13.974]  Not sure what his sister is doing.
[00:14.760 --> 00:16.130]  I think she just wants a cookie.
[00:17.561 --> 00:21.079]  But the mother's not paying attention
[00:21.460 --> 00:28.038]  to the water falling out of the sink, and she's drawing dishes.
[00:28.922 --> 00:29.600]  So I'm not sure.
[00:29.621 --> 00:29.916]  and she's...
[00:00.221 --> 00:07.560]  any water and there's a nice window but I don't see anything much else going on.
[00:00.060 --> 00:03.759]  Alright, we have a few more tasks to go through before we break for today.
[00:04.022 --> 00:04.329]  Okay.
[00:05.242 --> 00:07.474]  So, here's a picture.
[00:08.540 --> 00:10.931]  Just tell me everything that you see going on in this picture.
[00:12.740 --> 00:15.800]  Well there's a guy up in the tre

[00:01.409 --> 00:03.119]  I told you the stool was about to go over.
[00:04.265 --> 00:07.720]  And in a moment there's going to be real chaos, which will make what's going on in the picture
[00:07.861 --> 00:08.707]  look like nothing.
[00:10.320 --> 00:12.320]  The cookie jar is full, the lid is off the cookie jar.
[00:12.502 --> 00:14.160]  Do you want me to tell you all those things?
[00:14.644 --> 00:16.719]  Tell me all the things you see going on in the picture.
[00:16.920 --> 00:21.480]  The cabinet door has just swung open, the stool's about to fall, I guess I've told you
[00:21.560 --> 00:23.248]  that with the terrible crash.
[00:25.760 --> 00:28.860]  Mother is daydreaming, she doesn't even know what's going on behind her, I think that's
[00:28.961 --> 00:29.674]  very important.
[00:00.000 --> 00:00.883]  Sometimes it's difficult.
[00:08.510 --> 00:09.557]  Okay, that's fine.
[00:01.424 --> 00:04.493]  Okay, here's the picture. All the action you see going on.
[00:07.401 

[00:01.242 --> 00:13.000]  and she has her right finger up to her lip. There's a woman standing at the sink, drying
[00:13.100 --> 00:19.095]  dishes, and the sink is always flowing with water onto the floor.
[00:21.100 --> 00:23.657]  Um, you just left the accent.
[00:24.080 --> 00:24.243]  Mm-hmm.
[00:28.080 --> 00:29.080]  Left the accent.
[00:29.162 --> 00:29.223]  Bye.
[00:00.222 --> 00:01.717]  That water is coming out of the process.
[00:03.306 --> 00:03.449]  Okay.
[00:05.273 --> 00:05.960]  Good, thank you.
[00:01.624 --> 00:03.768]  The action that you see going on in that picture?
[00:08.900 --> 00:09.798]  Anything that I want.
[00:09.980 --> 00:14.315]  Okay, the boy is reaching for cookies and the stool is falling over.
[00:15.640 --> 00:19.114]  The girl is reaching for her brother to give her a cookie.
[00:20.620 --> 00:26.055]  Mother is doing dishes, but the water is spilling over on the floor.
[00:28.060 --> 00:28.836]  The windows open.
[00:14.358 --> 00:15.599]  Di

[00:02.096 --> 00:02.479]  That's it.
[00:03.730 --> 00:06.878]  There's a kid stealing cookies from the cookie jar.
[00:07.720 --> 00:09.473]  Stools about to topple over.
[00:10.320 --> 00:14.372]  His sister's asking for one.
[00:17.002 --> 00:18.078]  Cookie jar is open, of course.
[00:18.240 --> 00:19.025]  The cupboard's open.
[00:21.600 --> 00:22.639]  Mother's drying dishes.
[00:22.680 --> 00:24.055]  The sink is overflowing.
[00:25.765 --> 00:30.000]  There are some dishes on the sideboard, windows.
[00:00.160 --> 00:00.542]  open.
[00:07.220 --> 00:07.912]  What else you want?
[00:08.421 --> 00:09.653]  and curtains in the window.
[00:00.622 --> 00:04.475]  Why don't you take a look at this picture and tell me everything that you see going on in that picture.
[00:06.061 --> 00:08.000]  Everything that you see happening.
[00:08.060 --> 00:15.000]  Okay, the boy is standing up trying to get to the jar and the stool is falling.
[00:15.523 --> 00:17.960]  The girl is reaching up 

...

[00:00.040 --> 00:06.960]  water on the floor. Little boy is going to fall when that steps in that air. So, I think
[00:15.420 --> 00:21.237]  the cook jar is going to fall. And it's drying dishes and water faucets on, water is running
[00:23.143 --> 00:25.933]  the faucets on, the water's running all over the floor.
[00:05.952 --> 00:07.214]  What else am I slapping?
[00:10.280 --> 00:11.659]  Anything else?
[00:11.740 --> 00:12.855]  Just the things that you see happening.
[00:13.280 --> 00:19.537]  Yeah, the cookie jar, the stove, we're doing the dishes, the water's spilling.
[00:20.720 --> 00:20.802]  Okay.
[00:24.108 --> 00:24.269]  Okay.
[00:02.126 --> 00:04.192]  Tell me all the action going on in that picture.
[00:19.400 --> 00:22.512]  Can you tell me the action that you see going on in that picture?
[00:24.440 --> 00:26.016]  I'm going to do it.
[00:27.145 --> 00:28.836]  Somebody's going to dry them dishes.
[00:29.421 --> 00:30.000]  Okay, good.
[00:00.060 --> 00:08.973]  Wh

[00:02.070 --> 00:03.739]  Looks like somebody's laying out in the grass.
[00:04.503 --> 00:07.940]  Doesn't it?
[00:08.020 --> 00:11.698]  And the kid in the cookie jar in a tilted stool.
[00:13.293 --> 00:13.960]  What more do you want?
[00:14.245 --> 00:14.959]  That's fine.
[00:15.182 --> 00:16.434]  Water rolling on the floor?
[00:17.245 --> 00:17.837]  Okay.
[00:00.040 --> 00:05.339]  Okay? And I want you to tell me everything that you see going on in that picture.
[00:06.160 --> 00:08.799]  Just tell me everything that you see happening here.
[00:09.620 --> 00:10.396]  There's the picture.
[00:12.780 --> 00:14.720]  I'm supposed to tell you everything that's happening?
[00:14.742 --> 00:14.892]  Yeah.
[00:17.240 --> 00:20.559]  How many of this? One, two, three. Three women.
[00:22.720 --> 00:24.257]  There's some aprons on her.
[00:26.086 --> 00:28.896]  I'm going to push it back there.
[00:02.111 --> 00:02.693]  I love Hawaii.
[00:04.563 --> 00:07.296]  I love it.
[00:09.347 -

[00:02.048 --> 00:03.353]  dishes are on the same time
[00:07.903 --> 00:16.152]  the water is spilling out of the sink
[00:24.201 --> 00:27.720]  on several sites, just a very sections,
[00:28.204 --> 00:29.697]  primarily for anyone that,
Failed to align segment: original start time longer than audio duration, skipping...
[00:00.101 --> 00:03.176]  We'll have the new to see the outside world.
[00:00.161 --> 00:04.040]  Okay, the picture all you see going on in the picture, all right
[00:06.103 --> 00:08.940]  You tell me oh the boys in the cookie jar
[00:11.463 --> 00:12.897]  Handing a cookie to his sister
[00:16.321 --> 00:21.620]  Mother is drying dishes. Her sink is spilling over
[00:21.640 --> 00:21.841]  over.
[00:00.343 --> 00:01.798]  Anything else you see going on?
[00:02.081 --> 00:04.718]  The stool the boy is on is stilling.
[00:07.262 --> 00:09.336]  I said the sink's running over I think.
[00:19.531 --> 00:19.735]  No.
[00:20.223 --> 00:21.196]  Okay, that's fine.
[00:0

[00:02.972 --> 00:04.277]  And I don't know about the curtains.
[00:04.960 --> 00:10.173]  The curtains got different, something made there.
[00:13.000 --> 00:17.340]  She must have one of the double aprons back in life, isn't it?
[00:17.380 --> 00:18.709]  And the kids are at the cookie jar.
[00:20.360 --> 00:20.442]  Okay?
[00:21.727 --> 00:22.033]  Yeah.
[00:22.605 --> 00:22.768]  Okay.
[00:23.360 --> 00:23.708]  Okay.
[00:01.687 --> 00:04.076]  Well, my boy's trying to get some cookies.
[00:05.120 --> 00:10.955]  And the girl's holding her hand up because he has gotten one and she wants it.
[00:13.080 --> 00:17.433]  And his, where he's climbing up, is falling over.
[00:19.920 --> 00:27.018]  And this girl, she's washing dishes and she has something there for the dinner.
[00:27.820 --> 00:28.728]  And she spelt the...
[00:00.322 --> 00:02.780]  Water and it's all on the floor going
[00:09.304 --> 00:11.020]  She has a dishcloth in her hand
[00:18.603 --> 00:21.580]  And the boy he'

True

In [ ]:
name = "cn_segmented/adrso012_segment_2.wav"
whisperx_result = model.transcribe(name)
# load alignment model and metadata
model_a, metadata = whisperx.load_align_model(language_code=whisperx_result["language"], device=DEVICE)
# align whisper output
result_aligned = whisperx.align(whisperx_result["segments"], model_a, metadata, name, DEVICE)

[00:00.040 --> 00:06.759]  can't, well she'll feel it soon on her feet when the water hits her. That's what I see.


[]